# Train an ALPaCA dynamics model

Trains and saves an ALPaCA dynamics model on Rocker-bogie dynamics

In [1]:
import sys
sys.path.append('../')

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import matplotlib.pyplot as plt

# import dynamics (can replace with a different env if desired)
from src.envs.rocker_bogie_with_terrain_simplified import RockerBogieDynamics as Dynamics
from src.envs.rocker_bogie_with_terrain_simplified import state_space, action_space

# import dataloading tools
from src.models.dataset import TorchDatasetWrapper, DynamicsRandomDataset
from torch.utils.data import DataLoader

# import modeling tools
from src.models import Alpaca, AlpacaDynamics

### Define where to save the model

In [3]:
MODEL_PATH = "trained_models/rocker_bogie_alp_nom"


Initialize dynamics to train on

In [4]:
dynamics = Dynamics()

c rand
phi rand


Initialize nominal model

In [5]:
# These params shouldn't change
# Loose Sand parameters
# n = 1.1; k_c = 0.9e3; k_phi = 1523.4e3; k = 0.025; c1 = 0.18; c2 = 0.32 
# compact sand
n = 0.47; k_c = 0.9e3; k_phi = 1523.4e3; k = 0.038; c1 = 0.43; c2 = 0.32

c_placeholder = 0.9e3 
phi_placeholder = np.deg2rad(30) 
terr_params = [c_placeholder, phi_placeholder, n, k, k_c, k_phi, n, c1, c2] 
dt = 0.1

In [6]:
config = {
    'x_dim': dynamics.ob_dim,
    'u_dim': action_space.dimension,
    'learning_rate': 5e-5,
    'sigma_eps': [1e-2]*dynamics.ob_dim,
    'data_horizon': 40,
    'max_context': 10,
    'multistep_training': False,
    'learning_rate_decay': False,
    'phi_dim': 64,
    'learnable_noise':True,
    'bound_params':True
}

fnom = lambda x,u: dynamics.rover.get_fnom_function_vel_only(x, u, terr_params, dt)
phi_nom_fn=lambda x,u: dynamics.rover.get_c_tanphi_features_matrix_vel_only(x,u, terr_params, dt)
model = Alpaca(config, phi_nom_fn=phi_nom_fn, num_aug_features=2)
dynmodel = AlpacaDynamics(model, f_nom=fnom)


In [7]:
dataset = TorchDatasetWrapper(DynamicsRandomDataset(dynamics, state_space, action_space))
dataloader = DataLoader(dataset, batch_size=20)

Training progress is logged in the directory `runs` for visualization in Tensorboard

In [8]:
dynmodel.train(dataloader, 3, val_dataloader=None, verbose=True)
# dynmodel.train(dataloader, 3000, val_dataloader=None, verbose=True)

 33%|███▎      | 1/3 [00:00<00:01,  1.49it/s, tr_loss=14.234591]
Other loss =  7.289639  Ortho loss weighted= 0.0 Bounds loss weighted= 6.9449515
 67%|██████▋   | 2/3 [00:01<00:00,  1.50it/s, tr_loss=13.35361]
Other loss =  6.4204817  Ortho loss weighted= 0.0 Bounds loss weighted= 6.9331284
100%|██████████| 3/3 [00:02<00:00,  1.48it/s, tr_loss=13.338284]
Other loss =  6.419017  Ortho loss weighted= 0.0 Bounds loss weighted= 6.919267



In [9]:
dynmodel.save(MODEL_PATH)